In [1]:
# stdlib
from typing import Optional
from typing import Union

# third party
import numpy as np

# relative
from syft import PhiTensor
from syft import GammaTensor


class Activation(object):
    """Base class for activations.

    """
    def __init__(self):
        self.last_forward = None

    def forward(self, input: Union[PhiTensor, GammaTensor]):
        """Forward Step.

        Args:
            input (PhiTensor or GammaTensor): the input matrix
        """

        raise NotImplementedError

    def derivative(self, input: Optional[Union[PhiTensor, GammaTensor]]=None):
        """Backward Step.

        _extended_summary_

        Args:
            input (Optional[PhiTensor, GammaTensor], optional): If provide `input`, this function will not use `last_forward`. Defaults to None.
        """

        raise NotImplementedError

    def __str__(self):
        return self.__class__.__name__


class leaky_ReLU(Activation):

    def __init__(self, slope=0.01):
        super(leaky_ReLU, self).__init__()
        self.slope = slope

    def forward(self, input_array: Union[PhiTensor, GammaTensor]):
        # Last image that has been forward passed through this activation function
        self.last_forward = input_array

        gt = input_array > 0

        return gt * input_array + ((gt * -1) + 1) * input_array * self.slope

    def derivative(self, input_array: Optional[Union[PhiTensor, GammaTensor]] = None) -> Union[PhiTensor, GammaTensor]:
        last_forward = input_array if input_array else self.last_forward
        res = (last_forward > 0).child * 1 + (last_forward <= 0).child * self.slope 
        
        if isinstance(input_array, PhiTensor):
            return PhiTensor(child=res,
                           data_subjects=last_forward.data_subjects,
                           min_vals=last_forward.min_vals * 0,
                           max_vals=last_forward.max_vals * 1
                          )
        elif isinstance(input_array, GammaTensor):
            return GammaTensor(
                child=res,
                data_subjects=last_forward.data_subjects,
                min_val=last_forward.min_val * 0,
                max_val=last_forward.max_val * 1
            )
        else:
            raise NotImplementedError(f"Undefined behaviour for type {type(input_array)}")


/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from syft import nn

In [3]:
lr = nn.activations.leaky_ReLU()

In [4]:
from syft import DataSubjectList

sh = (5,5)

dsl = DataSubjectList(
    one_hot_lookup=np.arange(10),
    data_subjects_indexed=np.random.choice(np.arange(10), size=sh)
)
print(dsl.one_hot_lookup)
print(dsl.data_subjects_indexed)

[0 1 2 3 4 5 6 7 8 9]
[[2 5 9 2 2]
 [1 0 6 1 4]
 [0 5 5 6 0]
 [5 1 5 8 3]
 [8 5 3 6 8]]


In [5]:
from jax import numpy as jnp
import numpy as np
from syft.core.tensor.lazy_repeat_array import lazyrepeatarray as lra

gt = GammaTensor(
    child=jnp.array(2 * np.random.rand(*sh) - 1),
    data_subjects=dsl,
    min_val=lra(0, shape=sh),
    max_val=lra(1, shape=sh)
)

In [6]:
res = lr.forward(gt)

In [7]:
out = lr.derivative(res)

In [8]:
gt.child

DeviceArray([[ 0.38129884,  0.02295214, -0.38091889,  0.48091346,
              -0.96448819],
             [-0.58273043,  0.73720088, -0.70258983,  0.23396099,
              -0.89719281],
             [-0.0993434 , -0.37168033, -0.54137491,  0.85197807,
              -0.43752014],
             [ 0.84178948, -0.046312  ,  0.66395953,  0.01782525,
              -0.06668868],
             [-0.84651502, -0.54025007, -0.66932027,  0.3405847 ,
              -0.66528453]], dtype=float64)

In [9]:
out

GammaTensor(child=DeviceArray([[1.  , 1.  , 0.01, 1.  , 0.01],
             [0.01, 1.  , 0.01, 1.  , 0.01],
             [0.01, 0.01, 0.01, 1.  , 0.01],
             [1.  , 0.01, 1.  , 1.  , 0.01],
             [0.01, 0.01, 0.01, 1.  , 0.01]],            dtype=float64, weak_type=True), data_subjects=<syft.core.adp.data_subject_list.DataSubjectList object at 0x7f93701b58b0>, min_val=<lazyrepeatarray data: 0.0 -> shape: (5, 5)>, max_val=<lazyrepeatarray data: 1.01 -> shape: (5, 5)>, is_linear=True, func=<function no_op at 0x7f92a517dd30>, id='1629585384', state={})

In [10]:
gt > 0

GammaTensor(child=DeviceArray([[ True,  True, False,  True, False],
             [False,  True, False,  True, False],
             [False, False, False,  True, False],
             [ True, False,  True,  True, False],
             [False, False, False,  True, False]], dtype=bool), data_subjects=<syft.core.adp.data_subject_list.DataSubjectList object at 0x7f93701b58b0>, min_val=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_val=<lazyrepeatarray data: 1 -> shape: (5, 5)>, is_linear=True, func=<function GammaTensor.__gt__.<locals>._gt at 0x7f92a41a8c10>, id='1053572856', state={'1507121917': GammaTensor(child=DeviceArray([[ 0.38129884,  0.02295214, -0.38091889,  0.48091346,
              -0.96448819],
             [-0.58273043,  0.73720088, -0.70258983,  0.23396099,
              -0.89719281],
             [-0.0993434 , -0.37168033, -0.54137491,  0.85197807,
              -0.43752014],
             [ 0.84178948, -0.046312  ,  0.66395953,  0.01782525,
              -0.06668868],
         

In [11]:
gt.child > 1

DeviceArray([[False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False]], dtype=bool)

In [12]:
jnp.greater(gt.child, 1)

DeviceArray([[False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False]], dtype=bool)

In [13]:
gt > 1

GammaTensor(child=DeviceArray([[False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False],
             [False, False, False, False, False]], dtype=bool), data_subjects=<syft.core.adp.data_subject_list.DataSubjectList object at 0x7f93701b58b0>, min_val=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_val=<lazyrepeatarray data: 1 -> shape: (5, 5)>, is_linear=True, func=<function GammaTensor.__gt__.<locals>._gt at 0x7f92a41a83a0>, id='1654695923', state={'1507121917': GammaTensor(child=DeviceArray([[ 0.38129884,  0.02295214, -0.38091889,  0.48091346,
              -0.96448819],
             [-0.58273043,  0.73720088, -0.70258983,  0.23396099,
              -0.89719281],
             [-0.0993434 , -0.37168033, -0.54137491,  0.85197807,
              -0.43752014],
             [ 0.84178948, -0.046312  ,  0.66395953,  0.01782525,
              -0.06668868],
         